# Green Trap Analysis

Use this notebook to explore clustering and machine learning analyses on the processed WDI features.

In [ ]:
# --- CELL 1: Imports and Setup ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os

# Machine Learning & Clustering
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

# Statistics
from scipy import stats

# Panel Regression
try:
    from linearmodels import PanelOLS
except ImportError:
    print("⚠️ 'linearmodels' library not found. Installing...")
    !pip install linearmodels
    from linearmodels import PanelOLS

# Add 'src' to path
sys.path.append(os.path.abspath('../src'))
import config

# Plotting Configuration
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 7)
plt.rcParams['font.size'] = 11

print("✅ Libraries loaded. Ready for 5-Step Analysis Workflow!")

In [ ]:
# --- CELL 2: Load Data ---
try:
    df = pd.read_csv(config.PROCESSED_DATA_PATH)
    print(f"✅ Data Loaded: {df.shape[0]} rows, {df.shape[1]} columns")

    # Set Panel Index for later steps
    df_panel = df.set_index(['Country_Code', 'Year'])
    print("✅ Panel index set successfully.")

except FileNotFoundError:
    print(f"❌ Error: File not found at {config.PROCESSED_DATA_PATH}")
    print("   Please run 'python -m src.preprocessor' first.")

In [ ]:
# --- CELL 3: Step 1 - Descriptive Analysis ---
print("\n📊 STEP 1: DESCRIPTIVE ANALYSIS")

# 1. Turkey Time Series (Inflation vs. Renewables vs. Imports)
tur = df[df['Country_Code'] == 'TUR'].sort_values('Year')

fig, ax1 = plt.subplots(figsize=(14, 6))

# Area plot for Crisis Periods
# (Assuming 'Period' column exists from preprocessor)
crisis_years = tur[tur['Period'].isin(['Financial_Crisis', 'Pandemic', 'Energy_Crisis'])]['Year']
for year in crisis_years:
    ax1.axvspan(year-0.5, year+0.5, color='gray', alpha=0.15)

# Left Axis: Inflation
color1 = 'tab:red'
ax1.set_xlabel('Year')
ax1.set_ylabel('Inflation (CPI %)', color=color1)
ax1.plot(tur['Year'], tur['Inflation_CPI_Pct'], color=color1, linewidth=3, label='Inflation')
ax1.tick_params(axis='y', labelcolor=color1)

# Right Axis: Green & Energy
ax2 = ax1.twinx()
color2 = 'tab:green'
ax2.set_ylabel('Renewable Share (%) / Energy Imports (%)', color=color2)
ax2.plot(tur['Year'], tur['Renewable_Energy_Consumption_Pct'], color=color2, linestyle='--', linewidth=2, label='Renewable Share')
ax2.plot(tur['Year'], tur['Energy_Imports_Net_Pct'], color='tab:blue', linestyle=':', linewidth=2, label='Energy Import Dep.')
ax2.tick_params(axis='y', labelcolor=color2)

# Legends
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

plt.title("🇹🇷 Turkey: Inflation, Green Transition, and Energy Dependence (with Crisis Periods)")
plt.show()

# 2. Peer Comparison (Turkey vs. Green Leaders & Peers)
print("\n🌍 COMPARISON: Turkey vs. Peers")
peers = config.TURKEY_PEERS_EMERGING
leaders = ['DEU']
comparison_countries = ['TUR'] + peers + leaders

comp_df = df[df['Country_Code'].isin(comparison_countries)].copy()

plt.figure(figsize=(14, 6))
sns.lineplot(data=comp_df, x='Year', y='Inflation_CPI_Pct', hue='Country_Code',
             style='Country_Group', palette='tab10', linewidth=2)
plt.title("Inflation Trajectories: Turkey vs. Peers & Green Leaders")
plt.ylabel("Inflation (%)")
plt.show()

In [ ]:
# --- CELL 4: Step 2 - Clustering & PCA ---
print("\n🧩 STEP 2: CLUSTERING & PCA")

# 1. Feature Selection for Clustering
# We want to group countries by their "Green-Macro" profile
cluster_feats = [
    'Renewable_Energy_Consumption_Pct',
    'Inflation_CPI_Pct',
    'GDP_Growth_Pct',
    'Energy_Imports_Net_Pct',
    'Trade_Pct_GDP',
    'Energy_Intensity_Primary_MJ_Per_GDP'
]

# Prepare Data (Mean over last 5 years to get recent structural "types")
recent_df = df[df['Year'] >= 2018].groupby('Country_Code')[cluster_feats].mean().dropna()

# Scale Data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(recent_df)

# 2. Optimal K (Silhouette Score)
print("   Calculating Optimal K...")
scores = []
k_range = range(2, 8)
for k in k_range:
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    scores.append(silhouette_score(X_scaled, km.fit_predict(X_scaled)))

best_k = k_range[np.argmax(scores)]
print(f"   ✅ Optimal Number of Clusters (k): {best_k}")

# 3. Run K-Means
kmeans = KMeans(n_clusters=best_k, random_state=42, n_init=10)
recent_df['Cluster'] = kmeans.fit_predict(X_scaled)

# 4. PCA Visualization
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)
recent_df['PC1'] = X_pca[:, 0]
recent_df['PC2'] = X_pca[:, 1]

# Plot
plt.figure(figsize=(12, 8))
sns.scatterplot(data=recent_df, x='PC1', y='PC2', hue='Cluster', palette='viridis', s=200, style='Cluster')

# Annotate Countries
for country in recent_df.index:
    # Label Turkey, Peers, and Leaders
    if country == 'TUR' or country in config.TURKEY_PEERS_EMERGING or country in config.GREEN_LEADERS:
        x_pos = recent_df.loc[country, 'PC1']
        y_pos = recent_df.loc[country, 'PC2']
        weight = 'bold' if country == 'TUR' else 'normal'
        color = 'red' if country == 'TUR' else 'black'
        plt.text(x_pos+0.1, y_pos+0.1, country, color=color, weight=weight, fontsize=11)

plt.title(f"Country Clusters (PCA): Where does Turkey Fall? (k={best_k})")
plt.xlabel(f"PC1 ({pca.explained_variance_ratio_[0]:.1%} var)")
plt.ylabel(f"PC2 ({pca.explained_variance_ratio_[1]:.1%} var)")
plt.show()

# Show Turkey's Cluster Stats
tur_cluster = recent_df.loc['TUR', 'Cluster']
print(f"\n🇹🇷 Turkey is in Cluster {tur_cluster}")
print("Cluster Averages:")
print(recent_df.groupby('Cluster')[cluster_feats].mean().round(2))

In [ ]:
# --- CELL 5: Step 3 - Random Forest ---
print("\n🌲 STEP 3: RANDOM FOREST (Feature Importance)")

# 1. Define Features (12+ features as requested)
rf_features = [
    'Renewable_Energy_Consumption_Pct',
    'Fossil_Fuel_Consumption_Pct',
    'Energy_Imports_Net_Pct',
    'Energy_Intensity_Primary_MJ_Per_GDP',  # Efficiency
    'Broad_Money_Growth_Pct',               # Monetary
    'Real_Interest_Rate_Pct',               # Monetary
    'GDP_Growth_Pct',                       # Growth
    'Trade_Pct_GDP',                        # Openness
    'Current_Account_Balance_Pct_GDP',      # External Balance
    'Real_Effective_Exchange_Rate_Index',   # Competitiveness
    'Urban_Population_Pct',                 # Structural (if available)
    'Industry_Value_Added_Pct_GDP'          # Structural (if available)
]

# Check which exist in DF
available_feats = [f for f in rf_features if f in df.columns]
target = 'Inflation_CPI_Pct'

# 2. Train Model
rf_data = df[available_feats + [target]].dropna()
X = rf_data[available_feats]
y = rf_data[target]

rf = RandomForestRegressor(n_estimators=200, max_depth=15, random_state=42)
rf.fit(X, y)

# 3. Feature Importance Plot
importances = pd.Series(rf.feature_importances_, index=available_feats).sort_values(ascending=True)

plt.figure(figsize=(10, 8))
colors = ['green' if 'Renewable' in f or 'Energy' in f else 'gray' for f in importances.index]
importances.plot(kind='barh', color=colors)
plt.title("What Drives Inflation? (Green vs. Macro Factors)")
plt.xlabel("Importance Score")
plt.show()

print("Top 5 Drivers of Inflation:")
print(importances.sort_values(ascending=False).head(5))

In [ ]:
# --- CELL 6: Step 4 - Panel Regression (Manual) ---
print("\n📉 STEP 4: PANEL REGRESSION (Fixed Effects)")

# 1. Define Variables
target_var = 'Inflation_CPI_Pct'
explanatory_vars = [
    'Renewable_Energy_Consumption_Pct',
    'Energy_Imports_Net_Pct',
    'Broad_Money_Growth_Pct',
    'Current_Account_Balance_Pct_GDP'
]

# 2. Align Data (Crucial Step)
# We must drop missing values from specific columns first so X and y have matching indices
df_reg = df_panel[[target_var] + explanatory_vars].dropna()

# 3. Separate y and X
y_data = df_reg[target_var]
X_data = df_reg[explanatory_vars]

# 4. Run Model
# Note: PanelOLS(dependent, exog, ...) -> No "y=" or "X=" keywords allowed
model = PanelOLS(
    y_data,
    X_data,
    entity_effects=True,  # Country Fixed Effects
    time_effects=True     # Year Fixed Effects
)

results = model.fit(cov_type='clustered', cluster_entity=True)
print(results)

In [ ]:
# --- CELL 7: Step 5 - Heterogeneity Tests ---
print("\n🧪 STEP 5: HETEROGENEITY TESTS")

# ---------------------------------------------------------
# Test A: Does the effect differ for Energy Importers? (H2)
# ---------------------------------------------------------
print("\n--- A. Energy Importer Interaction ---")
# Create interaction term if missing
if 'Green_X_Importer' not in df_panel.columns:
    df_panel['Is_Energy_Importer'] = df_panel.index.get_level_values(0).isin(config.ENERGY_IMPORTERS).astype(int)
    df_panel['Green_X_Importer'] = df_panel['Renewable_Energy_Consumption_Pct'] * df_panel['Is_Energy_Importer']

vars_h2 = ['Renewable_Energy_Consumption_Pct', 'Green_X_Importer', 'Broad_Money_Growth_Pct', 'Energy_Imports_Net_Pct']
data_h2 = df_panel[['Inflation_CPI_Pct'] + vars_h2].dropna()

mod_h2 = PanelOLS(data_h2['Inflation_CPI_Pct'], data_h2[vars_h2], entity_effects=True, time_effects=True)
print(mod_h2.fit(cov_type='clustered', cluster_entity=True))


# ---------------------------------------------------------
# Test B: Temporal Dynamics (Structural vs Temporary) (H3)
# ---------------------------------------------------------
print("\n--- B. Temporal Dynamics (Pre-2020 vs Post-2020) ---")
data_pre = df_panel[df_panel.index.get_level_values('Year') < 2020]
data_post = df_panel[df_panel.index.get_level_values('Year') >= 2020]

# Run simple FE models for both periods
try:
    res_pre = PanelOLS(data_pre['Inflation_CPI_Pct'], data_pre[['Renewable_Energy_Consumption_Pct', 'Broad_Money_Growth_Pct']], entity_effects=True).fit()
    res_post = PanelOLS(data_post['Inflation_CPI_Pct'], data_post[['Renewable_Energy_Consumption_Pct', 'Broad_Money_Growth_Pct']], entity_effects=True).fit()

    coef_pre = res_pre.params['Renewable_Energy_Consumption_Pct']
    coef_post = res_post.params['Renewable_Energy_Consumption_Pct']

    print(f"Impact Pre-2020:  {coef_pre:.4f}")
    print(f"Impact Post-2020: {coef_post:.4f}")

    if coef_post > coef_pre:
        print("-> Conclusion: The inflationary impact has INTENSIFIED after 2020.")
    else:
        print("-> Conclusion: The impact is stable or decreasing.")
except Exception as e:
    print(f"Could not run temporal split: {e}")


# ---------------------------------------------------------
# Test C: Turkey vs. Specific Peers (Coefficient Comparison)
# ---------------------------------------------------------
print("\n--- C. Turkey vs. Peers (Sliding Window Correlation) ---")
# Since we can't easily run single-country regressions with FE,
# we compare the rolling correlation of Green Energy & Inflation
window = 5
plt.figure(figsize=(12, 6))

for country in ['TUR', 'POL', 'ZAF', 'MEX']:
    cty_data = df[df['Country_Code'] == country].set_index('Year').sort_index()

    # Calculate rolling correlation
    rolling_corr = cty_data['Renewable_Energy_Consumption_Pct'].rolling(window).corr(cty_data['Inflation_CPI_Pct'])

    style = '-' if country == 'TUR' else '--'
    width = 3 if country == 'TUR' else 1.5
    label = f"{country} (Turkey)" if country == 'TUR' else country

    plt.plot(rolling_corr.index, rolling_corr, linestyle=style, linewidth=width, label=label)

plt.axhline(0, color='black', linewidth=0.5)
plt.title(f"{window}-Year Rolling Correlation: Green Energy vs. Inflation")
plt.ylabel("Correlation Coefficient")
plt.legend()
plt.show()